In [1]:
import os
import sys 
sys.path.append('..')

import imp
import glob
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt

import utils.data_processor as data_processor
imp.reload(data_processor)
from utils.data_processor import *

from utils.train import *
from utils.model_criterion import *

import utils.crop as crop
imp.reload(crop)
from utils.crop import *

%config Completer.use_jedi = False

In [2]:
BRAIN_TYPE = 'whole'

CFG = '../cfgs/brain/brain_pospoolxyz.yaml' # 'cfgs/brain/brain_pointwisemlp.yaml'
config = config_seting(CFG)
torch.cuda.set_device("cuda:0") 

In [3]:
allowed_subjects = np.load('../raw_data/sub_with_all_data.npy', allow_pickle=True).tolist()

if not allowed_subjects:
    allowed_subdirs = [f"sub-{name[:-4]}" for name in os.listdir(f"../raw_data/normalized_label")]
else:
    allowed_subdirs = [f'sub-{name}' for name in allowed_subjects]
    
brains = [[f"../raw_data/output/fmriprep/{subdir}/anat/{name}" for name in os.listdir(f"../raw_data/output/fmriprep/{subdir}/anat") if name.endswith("Asym_desc-preproc_T1w.nii.gz")] for subdir in os.listdir(f"../raw_data/output/fmriprep/") if subdir.count(".") == 0 and subdir.startswith("sub") and subdir in allowed_subdirs]
brains = [item for sublist in brains for item in sublist]
brain_nums = [x.split('/')[4][4:] for x in brains]

if BRAIN_TYPE == 'whole':
    BRAIN_TYPE = 'full'
    
brains_loaded = [f"../dataset_ready_to_use/data_only_usefull_areas/{BRAIN_TYPE}/brains/{num}.npy" for num in tqdm(brain_nums)]
labels_loaded = [f"../dataset_ready_to_use/data_only_usefull_areas/{BRAIN_TYPE}/labels/{num}.npy" for num in tqdm(brain_nums)]
curv_loaded = [f"../dataset_ready_to_use/data_only_usefull_areas/{BRAIN_TYPE}/curv/{num}.npy" for num in tqdm(brain_nums)]
thickness_loaded = [f"../dataset_ready_to_use/data_only_usefull_areas/{BRAIN_TYPE}/thickness/{num}.npy" for num in tqdm(brain_nums)]
sulc_loaded = [f"../dataset_ready_to_use/data_only_usefull_areas/{BRAIN_TYPE}/sulc/{num}.npy" for num in tqdm(brain_nums)]

100%|██████████| 81/81 [00:00<00:00, 192486.47it/s]


In [4]:
train_dict = {
        'brains': brains_loaded[:20],
        'labels': labels_loaded[:20],
#         'curvs': curv_loaded[:20],
#         'thickness': thickness_loaded[:20],
#         'sulc': sulc_loaded[:20] 
    }

test_dict = {
        'brains': brains_loaded[20:30],
        'labels': labels_loaded[20:30],
#         'curvs': curv_loaded[20:30],
#         'thickness': thickness_loaded[20:30],
#         'sulc': sulc_loaded[20:30]
    }

In [5]:
train_dataset = BrainDataSegCrop(
        num_points=4096,
        task="train",
        crop_size=64,
        transforms=None,
        data_dict=train_dict,
        is_folded=True,
        return_min_point_train=True,
        return_abs_coords=True,
        return_pc_without_air_points=False,
        coin_flip_threshold=0.8,
    )

In [6]:
out = get_loader_crop(config=config,
                      batch_size=16,
                      num_points=4096,
                      train_dict=train_dict,
                      test_dict=test_dict,
                      crop_size=64,
                      is_folded=True,
                      return_abs_coords=True,
                      return_pc_without_air_points=False,
                      coin_flip_threshold = 0.8,
                      weighted_loss = True
                     )

Weights are: [0.01134033203125, 0.98865966796875]
MEANS are: {'brains': 145.58305}
STDS are: {'brains': 155.92049}
Train dataset created
Test dataset created


In [7]:
train_loader, test_loader, weights = out

In [10]:
validate_one_epoch(
    1,
    test_loader,
    model,
    criterion,
    config,
    num_votes=1,
    log_confusion_matrix=True,
    is_crop=True,
)

0.005981445311039686
0.011719971524291235
0.0059814453125
0.011891760708651862
0.011891760708651862


({'IoU': 0.0059814453125,
  'contrast': 3.084628787782882e-05,
  'dice': 0.011891760708651862,
  'AUC-ROC': 0.5476647778109155,
  'optimal cutoff': 0.5162228941917419,
  'TPR': 0.5387755102040817,
  'FPR': 0.4612305047279872,
  'loss': 0.7258202433586121},
 array([[    0, 40715],
        [    0,   245]]))

In [13]:
a = np.array([[    0, 40715],
        [    0,   245]])

In [15]:
a[1,0]

0

In [8]:
batch = next(iter(train_loader))

In [9]:
points, mask, points_labels, shape_labels = [batch[key] for key in ["current_points", "mask", "current_points_labels", "label"]]

In [10]:
features = points.transpose(1, 2).contiguous()
points = points[:, :, :3].cuda(non_blocking=True)
mask = mask.cuda(non_blocking=True)
features = features.cuda(non_blocking=True)
points_labels = points_labels.cuda(non_blocking=True)

In [8]:
model, criterion = build_multi_part_segmentation(config=config, weights=None, type='BCE')

In [9]:
model.cuda()
criterion.cuda()    

MultiShapeCrossEntropy(
  (bce): CrossEntropyLoss()
  (dice): DiceBCELoss(
    (bce): CrossEntropyLoss()
  )
)

In [13]:
pred = model(points, mask, features)

In [14]:
cr =  torch.nn.CrossEntropyLoss()

In [18]:
cr(pred[0], points_labels)

tensor(0.7544, device='cuda:0', grad_fn=<NllLoss2DBackward>)

In [19]:
criterion(pred, points_labels)

tensor(0.7544, device='cuda:0', grad_fn=<NllLossBackward>)

In [66]:
pred[0].shape

torch.Size([16, 2, 4096])

In [67]:
points_labels.shape

torch.Size([16, 4096])

In [79]:
points_labels.view(-1).shape

torch.Size([65536])

In [41]:
loss = nn.CrossEntropyLoss(reduction='none')
input = torch.randn((3, 2, 2046), requires_grad=True)
target = torch.empty((3, 2046), dtype=torch.long).random_(2)

In [42]:
loss(input, target)

tensor([[0.1249, 2.4720, 0.9472,  ..., 1.1377, 1.6171, 0.4655],
        [0.1198, 1.0488, 0.2479,  ..., 2.6044, 0.1698, 0.9088],
        [2.1431, 0.4324, 0.9785,  ..., 0.1881, 2.4278, 1.8509]],
       grad_fn=<ViewBackward>)